Thanks to all the contributors with kernels to this competition!

Specially, to the ones that provided code and comments that inspired us making this and other kernels we developed in the context of this competition:
https://www.kaggle.com/dmdm02/complete-eda-voting-lightgbm
https://www.kaggle.com/chocozzz/two-sigma-news-simple-eda-prophet-nlp
https://www.kaggle.com/ashishpatel26/bird-eye-view-of-two-sigma-nn-approach
https://www.kaggle.com/jsaguiar/baseline-with-news
https://www.kaggle.com/artgor/eda-feature-engineering-and-everything
https://www.kaggle.com/christofhenkel/market-data-nn-baseline
https://www.kaggle.com/smasar/tutorial-timeseriesapproach
https://www.kaggle.com/rabaman/0-64-in-100-lines
https://www.kaggle.com/guowenrui/market-nn-if-you-like-you-can-use-it-and-upvote/notebook
https://www.kaggle.com/smasar/eda-preprocessing-processing-evaluation

**Importamos librerías**

In [ ]:
import pandas as pd
import numpy as np
import time
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import collections

plt.style.use('seaborn')
sns.set(font_scale=2)

import warnings 
warnings.filterwarnings('ignore')
import os

Como se establecen en las normas de la competición, cargamos los datos usando la función que se ofrece para ello:  env.get_training_data() del paquete  twosigmanews .`


In [ ]:
from kaggle.competitions import twosigmanews
# You can only call make_env() once, so don't lose it!
env = twosigmanews.make_env()
print('Done!')

### Carga de los datasets

Cargamos datos de entrenamiento del fichero de datos de mercado y del fichero de datos de noticias

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

In [ ]:
print("In total: ", market_train_df.shape)

In [ ]:
#market_train_df.head()
market_train_df.tail()

In [ ]:
print("In total: ", news_train_df.shape)

In [ ]:
#news_train_df.head()
news_train_df.tail()


Cargamos datos de test del fichero de datos de mercado y del fichero de datos de noticias

In [ ]:
days = env.get_prediction_days()
(market_test_df, news_test_df, predictions_template_df) = next(days)

In [ ]:
print("In total: ", market_test_df.shape)

In [ ]:
market_test_df.head()

In [ ]:
print("In total: ", news_test_df.shape)

In [ ]:
news_test_df.head()

In [ ]:
predictions_template_df.head()

In [ ]:
print("In market_train_df: ", market_train_df.shape);print("In market_test_df: ", market_test_df.shape);
print("In news_train_df: ", news_train_df.shape);print("In news_test_df: ", news_test_df.shape)

- El fichero de entrenamiento de datos de mercado, tiene  **4.07 millones de registros ( 2007-02-01 ~ 2016-12-30 )** , y el de entrenamiento tiene  **1800 registros (un día de cotización, el  2017-01-03 )**. El fichero de noticias son más registros en esos mismos periodos , ya que hay más de una noticia por activo 
- El fichero de test, no tiene los campos`returnsOpenNextMktres10` (variable objetivo)  y` universe`, por las reglas definidas para la  competición. En nuestro caso este fichero no lo usaremos para la validación del modelo, ya que no tenemos intención de submitir para la competición, ni se puede por estar fuera de plazo. La submisión es condición necesaria para validar los datos en test. 


### Dadas las características del proyecto, de cara a poder medir la accuracy, simplemente consultamos las características de los ficheros de test... pero no vamos a usarlos en lo sucesivo, sino que vamos a coger un subconjunto de los ficheros de train, que tengan las mismas características: los datos de mercados del último día y los datos del fichero de noticias posteriores a las 22h del penúltimo día (más todos los del último día).

In [ ]:
del market_test_df, news_test_df

start = '2016-12-29 22:00:00+00:00'

market_test_df = market_train_df.loc[market_train_df['time'] > start]
news_test_df = news_train_df.loc[news_train_df['time'] > start]

market_train_df = market_train_df.loc[market_train_df['time'] <= start].reset_index(drop=True)
news_train_df = news_train_df.loc[news_train_df['time'] <= start].reset_index(drop=True)

## Análisis exploratorio univariante de todas las variables

 ### Comprobamos la existencia de datos nulos

#### Market data

In [ ]:
percent = (100 * market_train_df.isnull().sum() / market_train_df.shape[0]).sort_values(ascending=False)
percent.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Value Percent(%)", fontsize = 20)
plt.title("Total Missing Value by market_train_df", fontsize = 20)

In [ ]:
percent1 = (100 * market_test_df.isnull().sum() / market_test_df.shape[0]).sort_values(ascending=False)
percent1.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Value Percent(%)", fontsize = 20)
plt.title("Total Missing Value by market_test_df", fontsize = 20)

Vemos que las variables para las que hay nulos, tanto en el fichero de train como en el de test son las variables con los valores ajustados al mercado para los returns-returnsOpenPrevMktres10, returnsClosePrevMktres10, returnsOpenPrevMktres1 y returnsClosePrevMktres1. 

Las variables son las mismas y los porcentajes son parecidos en ambos data sets:

- **market_train_df** : { returnsOpenPrevMktres10 : 2.284680 , returnsClosePrevMktres10 : 2.283599, returnsOpenPrevMktres1 : 0.392540 , returnsClosePrevMktres1 : 0.392344 }

- **market_test_df** : { returnsClosePrevMktres10 : 2.029622, returnsOpenPrevMktres10 : 2.029622, returnsOpenPrevMktres1 : 0.658256, returnsClosePrevMktres1 : 0.658256 }


#### News data

In [ ]:
percent = (100 * news_train_df.isnull().sum() / news_train_df.shape[0]).sort_values(ascending=False)
percent.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Value Percent(%)", fontsize = 20)
plt.title("Total Missing Value by news_train_df", fontsize = 20)

In [ ]:
percent1 = (100 * news_test_df.isnull().sum() / news_test_df.shape[0]).sort_values(ascending=False)
percent1.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Value Percent(%)", fontsize = 20)
plt.title("Total Missing Value by news_test_df", fontsize = 20)

Aparentemente no hay nulos en el fichero de news, pero en realidad, están disimulados.

Veamos:

In [ ]:
news_train_df['headlineTag'].unique()[0:5]

'' no es reconocido como nulo, así que hay que hacer la transformación

In [ ]:
for i in news_train_df.columns.values.tolist():
    news_train_df[i] = news_train_df[i].replace('', np.nan)  

news_train_df['headlineTag'].unique()[0:5]

In [ ]:
for i in news_test_df.columns.values.tolist():
    news_test_df[i] = news_test_df[i].replace('', np.nan)

In [ ]:
percent = (100 * news_train_df.isnull().sum() / news_train_df.shape[0]).sort_values(ascending=False)
percent.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Value Percent(%)", fontsize = 20)
plt.title("Total Missing Value by news_train_df", fontsize = 20)

In [ ]:
percent1 = (100 * news_test_df.isnull().sum() / news_test_df.shape[0]).sort_values(ascending=False)
percent1.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Value Percent(%)", fontsize = 20)
plt.title("Total Missing Value by news_test_df", fontsize = 20)

#### Vemos como el fichero de news tiene nulos en headlineTag (el 70% de los registros) y el headline

### Comprobar valores únicos


#### Market

In [ ]:
totalNumber = (market_train_df.nunique()).sort_values(ascending=False)
totalNumber.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Unique Number", fontsize = 20)
plt.title("Unique Number by market_train_df", fontsize = 20)

Vemos cómo, las variables de los returns, así como el volume, tienen una variedad de valores muy alta, como es lógico. Los valores para open y close también presentan un gran número de posibles valores (aunque menor). Vamos a restringuirnos al resto de columnas para que el gráfico sea representativo.

In [ ]:
market_train_df2 = market_train_df.drop(columns=["returnsOpenNextMktres10","returnsOpenPrevMktres10","returnsClosePrevMktres10","returnsOpenPrevMktres1","returnsClosePrevMktres1","returnsOpenPrevRaw10","returnsClosePrevRaw10","returnsOpenPrevRaw1","returnsClosePrevRaw1","volume","close","open"])
totalNumber = (market_train_df2.nunique()).sort_values(ascending=False)
totalNumber.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Unique Number", fontsize = 20)
plt.title("Unique Number by market_train_df", fontsize = 20)

In [ ]:
market_train_df2.nunique()

In [ ]:
market_test_df2 = market_test_df.drop(columns=["returnsOpenNextMktres10","returnsOpenPrevMktres10","returnsClosePrevMktres10","returnsOpenPrevMktres1","returnsClosePrevMktres1","returnsOpenPrevRaw10","returnsClosePrevRaw10","returnsOpenPrevRaw1","returnsClosePrevRaw1","volume","close","open"])
totalNumber = (market_test_df2.nunique()).sort_values(ascending=False)
totalNumber.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Unique Number", fontsize = 20)
plt.title("Unique Number by market_test_df", fontsize = 20)

In [ ]:
market_test_df2.nunique()

Los datos nos muestran que:

- En train tenermos cotizaciones de casi 2500 días, mientras que en test tenemos los datos de 1 sólo día
- Los posibles valores para la variable universe son 2 (encaja con la descripción de la variable: Sí cotiza / No cotiza), tanto en train como en test.
- En train tenemos datos de casi 3800 activos, mientras que en test tenemos datos de algo más de 1800.
- En train tenemos datos de algo más de 3500 compañías, mientras que en test estamos cerca de las 1800.


### La principal conclusión que se puede extraer es que la inmensa mayoría de las compañias (casi el 95%) tienen únicamente un activo relacionado... o, en términos de las variables: la inmensa mayoría de assetNames están relacionados con un único assetCode.

#### News

In [ ]:
percent2 = (news_train_df.nunique()).sort_values(ascending=False)
percent2.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Unique Number", fontsize = 20)
plt.title("Unique Number by news_train_df", fontsize = 20)

En este fichero también hay una serie de variables que tienen tal número de ocurrencias que no permiten ver el número de ocurrencias de las demás en el gráfico. Prescindimos de ellas para poder ver al resto.

In [ ]:
news_train_df2 = news_train_df.drop(columns=["sourceId","headline","time","sourceTimestamp","firstCreated","subjects","sentimentNegative","sentimentPositive","sentimentNeutral","audiences","bodySize"])
totalNumber = (news_train_df2.nunique()).sort_values(ascending=False)
totalNumber.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Unique Number", fontsize = 20)
plt.title("Unique Number by news_train_df", fontsize = 20)

In [ ]:
news_train_df2.nunique()

In [ ]:
news_test_df2 = news_test_df.drop(columns=["sourceId","headline","time","sourceTimestamp","firstCreated","subjects","sentimentNegative","sentimentPositive","sentimentNeutral","audiences","bodySize"])
totalNumber = (news_test_df2.nunique()).sort_values(ascending=False)
totalNumber.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Unique Number", fontsize = 20)
plt.title("Unique Number by news_test_df", fontsize = 20)

In [ ]:
news_test_df2.nunique()

En el fichero de noticias podemos volver a confirmar cómo el número de assetCodes y de assetNames están en el mismo orden de magnitud (en el fichero de test, de hecho, coinciden). 

Por otro lado, vemos cómo en el caso del fichero de noticias, las noticias no son por día, sino que hay noticias hasta un desglose de tiempo del timestamp.

**Estas informaciones son relevantes a la hora de plantearnos los criterios por los que podemos unir ambos ficheros:**

* **En principio, podríamos hacer el cruce tanto por assetCode como por assetName**
* **A la hora de unir por tiempo, teniendo también en cuenta las características del fichero de test original (que hemos heredado al fichero de test que vamos a usar), habría que un registro del fichero de market (sólo hay un registro por activo y día) tiene correspondencia con todos los registros existentes en el fichero de news (pueden haber 0, 1 o varios) entre las 22h del día anterior y las 22h de dicho día.**

## 5.1.2 Variable por variable

### Fichero Market

* **TIME**

El día de cotización, representado en formato timestamp, siempre a las 22h.

In [ ]:
market_train_df.time.describe()

Vemos que los más de 4 millones de registros corresponden a algo menos de 2500 días
El día que tiene más registros (el día en que cotizaron más activos) es el 13 de octubre de 2016, con 1854.

Veámos la distribución de registros (activos cotizados) por día gráficamente.

In [ ]:
fig,axes = plt.subplots(1,1,figsize=(15,10))
axes.set_title("Time Distro")
axes.set_ylabel("# of records")
axes.set_xlabel("date")
axes.plot(market_train_df.time.dt.date.value_counts().sort_index().index, market_train_df.time.dt.date.value_counts().sort_index().values)

Comprobamos cómo el número de activos cotizados ha ido creciendo en el periodo de estudio, aunque con altibajos:

- Se comprueba una gran caída durante 2009, que se recupera durante 2010.
- Vuelve a haber una tendencia descendiente durante 2013 y 2014.
- Hay una caída inusual a finales de 2014, que parece que puede deberse a error en los datos (no parece normal que haya tan pocos activos cotizando de forma tan repentina).
- La tendencia vuelve a ser ascendiente en 2015 y 2016, hallando el máximo en octubre de 2016.

Haremos foco en esa caída inusual a finales de 2014:

In [ ]:
market_train_df.time.value_counts()

Parece estar centrada fundamentalmente en octubre de 2014, veámoslo gráficamente:

In [ ]:
datefrom = '2014-09-15 22:00:00+00:00'
dateto = '2014-11-30 22:00:00+00:00'

market_train_df1 = market_train_df.loc[(market_train_df['time'] > datefrom)&(market_train_df['time'] < dateto)]

fig,axes = plt.subplots(1,1,figsize=(15,10))
axes.set_title("Time Distro")
axes.set_ylabel("# of records")
axes.set_xlabel("date")
axes.plot(market_train_df1.time.dt.date.value_counts().sort_index().index, market_train_df1.time.dt.date.value_counts().sort_index().values)

Resulta muy extraño que:
- viniendo de valores por encima de 1550 activos por sesión, entre finales de septiembre y principios de octubre, de un día a otro hay una bajada de mas de 200 registros
- se mantiene un nivel en torno a 1350 durante todo el mes de octubre y principios de noviembre
- y, a principios de noviembre, de un día para otro, hay una subida de 300, para matenerse ya luego en rangos por encima de 1650.

Podría ser que haya algún problema de muestreo relacionado con esos días.

* **ASSETCODE**

 Identificador único de cada activo.

In [ ]:
market_train_df.assetCode.describe()

Vemos que hay un total de 3780 activos en el fichero.
El que aparece más veces (el que más días ha cotizado en el periodo) es AU.N, 2497 veces... lo que quiere decir que ha cotizado todos los días del periodo.

In [ ]:
market_train_df.assetCode.value_counts().describe()

Mirando la distribución de la variable, vemos cómo:

- la media de días que cotiza un activo en el periodo es de 1077 días (algo más del 40% de los días).
- hay un activo que ha cotizado un sólo día, mientras que otro (AU.N) ha cotizado todos los días.
- el 50% de los activos ha cotizado menos de 805 días (algo menos de un tercio de los días).
- el 75% de los activos ha cotizado menos de 1988 días (algo menos del 80% de los días).

In [ ]:
market_train_df['assetCode'].value_counts()

Comprobamos cómo AU.N no es el único activo que ha cotizado todos los días...
y cómo hay 10 activos que tan sólo han cotizado un día.

* **ASSETNAME**

Es la compañía a la que pertenece uno o más activos (assetcodes).
En el caso de que el activo no tenga registros en el fichero de noticias, el assetname aparece como "Unknown".

In [ ]:
market_train_df.assetName.describe()


Vemos cómo hay 3508 valores distintos para el assetname (número de compañías que han cotizado en el periodo) y que la que más veces aparece es "Unknown".

Esto quiere decir que en realidad han cotizado más compañías, pero sólo 3507 de ellas tienen reflejo en el fichero de noticias.

Ahondaremos más en la relación entre assetcode y assetname en el análisis relacional entre variables

Apoyándonos en un WordCloud podemos ver las compañías que más representadas están en el fichero.

In [ ]:
from wordcloud import WordCloud
# Create the wordcloud object
wordcloud = WordCloud(width=1024, height=1024, margin=0).generate(" ".join(market_train_df.assetName))
 
# Display the generated image:
fig,ax = plt.subplots(1,1,figsize=(20,10))
ax.imshow(wordcloud, interpolation='bilinear')
ax.axis("off")
ax.margins(x=0, y=0)
plt.show()

Las compañías que más aparecen son: Group Inc, Holdings Inc, International Inc, Financial Corp, Systems Inc, Industries Inc, Pharmaceuticals Inc, Services Inc, Technologies, Inc, Resources Inc, ...

* **UNIVERSE**

Indicador booleano sobre si el activo será incluido en el scoring de ese día. 
El valor no se proporciona en el conjunto de test real, pero sí que estará en el conjunto de test que hemos escogido finalmente (el último día de training). 
El universo de trading en un día es el conjunto de activos que están disponibles para trading en ese día 
(la función de scoring no considerará activos que estén fuera del universo de trading). 
El universo de trading cambia a diario.

In [ ]:
market_train_df.universe.value_counts()

Vemos cómo prácticamente el 60% de los registros forman parte del universo de trading.

Ahora, gráficamente:

In [ ]:
univers_df_dict = dict(collections.Counter(list(market_train_df.universe)))
percent_univers_df_dict = {k: v / total for total in (sum(univers_df_dict.values()),) for k, v in univers_df_dict.items()}
explode=(0,0.1)
labels ='notUniverse','isUniverse'
fig, ax = plt.subplots(1,1, figsize=(8,8))
ax.set_title("Univere Status")
ax.pie(list(percent_univers_df_dict.values()), explode=explode, labels=labels, autopct='%1.1f%%',shadow=True, startangle=90)


In [ ]:
market_train_df['universe'].plot.hist(title = 'universe Histogram');
plt.xlabel('universe');

Ahondaremos más en el análisis de esta variable en el apartado de análisis relacional.

Podríamos coger un subconjunto de registros con valor 0 en esta variable y ver los valores de las variables open y close.
Habría que ver qué pasa con los valores de las acciones entre día anterior y día posterior... y, si el día con 0 no varía en absoluto, podría tener sentido eliminar estos registros.

* **VOLUME**

volúmen de acciones negociado en el día para el activo.

In [ ]:
market_train_df.volume.describe()

Vemos como la desviación estándar parece bastante significativa.
Llama la atención el valor máximo.

Veamos gráficamente la distribución de valores en esta variable:

In [ ]:
fig, axes = plt.subplots(figsize=(20,10))
axes.set_title("Volume")
axes.set_ylabel("volume")
axes.set_xlabel("records")
axes.plot(market_train_df["volume"])

Llaman la atención algunos picos que se ven en la gráfica. Parece haber ciertos atípicos.

Por la variable en sí misma no se puede decir demasiado... puede que en el análisis relacional con otras variables extraigamos más.
Lo mismo se puede ver a qué activos corresponden estos atípicos y ver si puede haber noticias relacionadas que justificasen el inusual volúmen de negociación.

* **CLOSE**

Precio al cierre (no ajustado por splits ni dividendos)

In [ ]:
market_train_df.close.describe()

In [ ]:
fig, axes = plt.subplots(figsize=(20,10))

axes.set_title("Close Price")
axes.set_ylabel("close price")
axes.set_xlabel("records")
axes.plot(market_train_df["close"])

Por la variable en sí misma no se puede decir demasiado... puede que en el análisis relacional con otras variables extraigamos más.
Lo mismo se puede ver la evolución del valor para ciertos activos.

* **OPEN**

Precio de apertura (no ajustado por splits ni dividendos)

In [ ]:
market_train_df.open.describe()

Llama la atención el máximo que parece completamente de otro mundo.

Veamos gráficamente la distribución de la variable:

In [ ]:
fig, axes = plt.subplots(figsize=(20,10))
axes.set_title("Open Price")
axes.set_ylabel("open price")
axes.set_xlabel("records")
axes.plot(market_train_df["open"])

Llaman la atención dos picos que hacen que el resto de la gráfica no pueda verse correctamente...
podríamos eliminar ese par de picos, para ver la gráfica bien:

In [ ]:
atypical = 2000

market_train_df3 = market_train_df.loc[market_train_df['open'] < atypical]

fig, axes = plt.subplots(figsize=(20,10))
axes.set_title("Open Price")
axes.set_ylabel("open price")
axes.set_xlabel("records")
axes.plot(market_train_df3["open"])

Vemos cómo la distribución es muy parecida a la de la variable close, lo cual es completamente lógico.

No obstante, se puede ver cómo, aparte de los dos atípicos que hemos eliminado hay algunos picos que también parecen extraños.
Dejamos para el análisis relacional analizar esos picos, en comparación con los valores de close...
puede que se trate de valores erróneos que haya que eliminar del dataset

* **RETURNS**

Los retornos están calculados respecto a la apertura y el cierre, para distintos intervalos de tiempo (1 y 10 días), tanto en crudo como ajustados al mercado.
Todos ellos son retornos respecto al pasado, salvo la variable objetivo que es el retorno a 10 días en el futuro. 

In [ ]:
market_returns_df = pd.concat(
    [
        market_train_df["returnsClosePrevRaw1"].describe(),
        market_train_df["returnsOpenPrevRaw1"].describe(),
        market_train_df["returnsClosePrevMktres1"].describe(),
        market_train_df["returnsOpenPrevMktres1"].describe(),
        market_train_df["returnsClosePrevRaw10"].describe(),
        market_train_df["returnsOpenPrevRaw10"].describe(),
        market_train_df["returnsClosePrevMktres10"].describe(),
        market_train_df["returnsOpenPrevMktres10"].describe(),
        market_train_df["returnsOpenNextMktres10"].describe()
        ],
        axis=1
    )
market_returns_df

Vemos cómo:

- hay valores negativos para el primer cuartil de cualquiera de los periodos
- en el caso de los valores ajustados (tanto al cierre como de apertura) comparando con el día anterior, son negativos más de la mitad de los retornos (el percentil 50% es negativo)
- la varianza de los valores de apertura es siempre mayor (100 veces) que la de los valores de cierre. Esto puede deberse a los atípicos que hemos observado en la variable OPEN.
- los valores en crudo y ajustados de un mismo periodo temporal se mueven en las mismas magnitudes (por regla general)
- los valores de los retornos en los diez últimos días y en los diez siguientes días son prácticamente iguales. Esto induce a pensar que las variables correspondientes a retornos en los diez últimos días (tanto en crudo, como sobre todo ajustados) puede ser un buen predictor de los retornos para dentro de diez días. Podría ser que las desviaciones de esta norma se deban precisamente al efecto de las noticias. Esto será algo que intentaremos analizar en el apartado de análisis relacional entre variables. 